# How do acquisitions change over the next 3 months?
- search by each month

In [1]:
import geopandas as gpd
import requests
import json
import csv
import os
import datetime
import pandas as pd
import time

# Helper function to printformatted JSON using the json module
def p(data):
    print(json.dumps(data, indent=2))

In [2]:
with open('/home/jovyan/capella/credentials.json') as f: # USER: Input path to credentials.json file.
    data = json.load(f)
    username = data['username']
    password = data['password']

In [3]:
# Get a valid token from the auth service
r = requests.post("https://api.capellaspace.com/token", 
                  headers = {'Content-Type': 'application/x-www-form-urlencoded'}, auth=(username,password))
access_token = r.json()["accessToken"]
# p(accessToken)

# GET user ID and org ID
headers = {'Authorization':'Bearer ' + access_token}
r = requests.get("https://api.capellaspace.com/user", headers=headers)
user_id = r.json()["id"]
org_id = r.json()["organizationId"]
#p(r.json())

# Print user and org ID
print('User email: ', r.json()['email'], '\nOrganization: ', r.json()['organization']['name'], '\nEnvironment: ', r.json()['apiEnvironmentRole'])

User email:  scottyh@uw.edu 
Organization:  University Of Washington, Seattle-NASA CSDA 
Environment:  production


In [4]:
# NOTE: use 3D coordinates for rugged terrain / volcanoes
point = {'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'properties': {},
   'geometry': {'type': 'Point',
    'coordinates': (-121.744564, 46.796787, 1589.0)}}]}

In [5]:
aoi_geom = point['features'][0]['geometry']
aoi_geom

{'type': 'Point', 'coordinates': (-121.744564, 46.796787, 1589.0)}

In [6]:
# Name and description
request_name = 'Rainier - Paradise - 45' # USER: Set the name of the request.
request_description = 'Mt. Rainier Paradise repeatdaily' # USER: Set a description for the request.

#NOTES
orbital_plane = [45] # options: 45, 53, 97 # only 97 will have polar coverage
orbit_state = "either" # options: ascending, descending, either
look_direction = "either" # options: left, right, either
look_angle_min = 25 # full: 5, extended: 15, standard: 25
look_angle_max = 50 # max: 50

# Footprint size guidelines:
# - Spotlight: length = 5000, width = 5000
# - Sliding Spotlight: length = 10000, width = 5000
# - Stripmap: length = 20000, width = 5000-10000

# STRIP
img_width = 5000 # in meters
img_length = 20000 # in meters

# SLIDE
#img_width = 5000 # in meters
#img_length = 10000 # in meters

# SPOT
#img_width = 5000 # in meters
#img_length = 5000 # in meters

In [7]:
def submit_request(start_date=datetime.datetime.utcnow(), end_date=None):
    ''' get access estimated for start+30 days'''
    # Access request window
    # Default: 30-day window starting at current UTC date/time
    # MAX 90-days out
    ts90 = pd.Timestamp.today('UTC') + pd.Timedelta('89D')

    if end_date == None:
        end_date = start_date + datetime.timedelta(days = 30) # 31 days or less!!!
    elif end_date >= ts90:
        print('setting max window close (today + 90 days)')
        end_date = ts90

    # USER: To change the window from the default, uncomment and set the start_date and end_date for your desired window in UTC in YYYY,M,D format.
    # start_date = datetime.date(YYYY,M,D)
    # end_date = datetime.date(YYYY,M,D)
    window_open = start_date.strftime("%Y-%m-%dT%H:%M:%SZ") # format start date
    window_close = end_date.strftime("%Y-%m-%dT%H:%M:%SZ") # format end date
    print(f'Searching {window_open} To {window_close}')
    
    # Define access request
    access_request = {
        "geometry": aoi_geom,
        "properties": {
          "userId": user_id,
          "orgId": org_id,
          "accessrequestDescription": request_description,
          "accessrequestName": request_name,
          "windowOpen": window_open,
          "windowClose": window_close,
          "accessConstraints": {
              "ascDsc": orbit_state,
              "lookDirection": look_direction,
              "orbitalPlanes": orbital_plane,
              "offNadirMin": look_angle_min,
              "offNadirMax": look_angle_max, #offNadirMax #seems possibly stuck on 40?
              "imageWidth": img_width,
              "imageLength": img_length
          }
        },
        "type": "Feature"
    }

    # POST request
    headers = {'Authorization':'Bearer ' + access_token}
    r = requests.post("https://api.capellaspace.com/ma/accessrequests/", json = access_request, headers = headers)
    #p(r.json())
    return r

In [8]:
start = pd.Timestamp.today()
end = pd.Timestamp('2023-12-01T00:00:00Z')
r1 = submit_request(start, end)

Searching 2023-11-14T05:27:28Z To 2023-12-01T00:00:00Z


In [9]:
# Results retrieved via access request ID:
ar_id = r1.json()['properties']['accessrequestId']
ar_id

'43184453-3598-47cd-8de9-b7b760b27b4f'

In [10]:
# Check after a bit
status = requests.get("https://api.capellaspace.com/ma/accessrequests/" + ar_id, headers = headers)
#status.json() # See processing: 'completed'

In [11]:
# GET access requests
time.sleep(5) #takes a little time to post
results = requests.get("https://api.capellaspace.com/ma/accesses/" + ar_id, headers=headers)

# View requests as table
df1 = pd.json_normalize(results.json())
col_interest = ['spacecraftId', 'windowOpen', 'windowClose', 'orbitalPlane', 'accessProperties.ascdsc', 
                'accessProperties.lookDirection', 'accessProperties.offNadirMin']

print('Number of possible acquisitions:', len(df1))
df1[col_interest].head(3) 

Number of possible acquisitions: 82


,spacecraftId,windowOpen,windowClose,orbitalPlane,accessProperties.ascdsc,accessProperties.lookDirection,accessProperties.offNadirMin
0,10,2023-11-14T16:19:37.893Z,2023-11-14T16:21:29.816Z,45,ascending,left,46.630864
1,9,2023-11-14T16:35:42.780Z,2023-11-14T16:38:22.017Z,45,ascending,left,41.240362
2,10,2023-11-14T17:59:35.802Z,2023-11-14T18:01:52.407Z,45,ascending,left,32.691706


In [12]:
df1 = df1.sort_values('windowOpen')
df1.head(3)

,center,centerEcef,spacecraftId,accessId,accessrequestId,tileId,windowOpen,windowClose,windowOpenLocal,windowCloseLocal,orbitalPlane,accessProperties.ascdsc,accessProperties.lookDirection,accessProperties.localTime,accessProperties.azimuthOpen,accessProperties.azimuthClose,accessProperties.elevationMin,accessProperties.elevationMax,accessProperties.offNadirMin,accessProperties.offNadirMax
0,"[-121.7445629415389, 46.79641971050866, 1589]","[-2301990.664414201, -3720764.479257549, 46274...",10,b87fe4e3-e320-4e83-af30-b4abbfe77319,43184453-3598-47cd-8de9-b7b760b27b4f,951dcb65-990d-496e-95bb-6297d54f9dbb,2023-11-14T16:19:37.893Z,2023-11-14T16:21:29.816Z,2023-11-14T08:19:37.893Z,2023-11-14T08:21:29.816Z,45,ascending,left,29597,188.751483,130.429458,33.178852,37.414813,46.630864,49.999467
4,"[-121.74456155035168, 46.79641971868754, 1589]","[-2301990.5737222885, -3720764.534587969, 4627...",10,c557bcc1-fd36-48aa-8e3b-4436548be780,43184453-3598-47cd-8de9-b7b760b27b4f,e73a63de-f7b9-49b1-be01-cabb3e7de083,2023-11-14T16:19:37.893Z,2023-11-14T16:21:29.816Z,2023-11-14T08:19:37.893Z,2023-11-14T08:21:29.816Z,45,ascending,left,29597,188.751492,130.429465,33.178851,37.414815,46.630862,49.999467
1,"[-121.74456289285206, 46.79641971068774, 1589]","[-2301990.661244861, -3720764.4812013116, 4627...",9,b0b80fb9-f46e-4e2e-8a25-146fe51f01a0,43184453-3598-47cd-8de9-b7b760b27b4f,34ddc2b4-7cce-42dc-bdf5-80ea22f83e5f,2023-11-14T16:35:42.780Z,2023-11-14T16:38:22.017Z,2023-11-14T08:35:42.780Z,2023-11-14T08:38:22.017Z,45,ascending,left,30585,208.307328,118.874593,33.275784,43.986568,41.240362,49.999704


In [13]:
# Seems there are duplicated entries
df1[:2].T

,0,4
center,"[-121.7445629415389, 46.79641971050866, 1589]","[-121.74456155035168, 46.79641971868754, 1589]"
centerEcef,"[-2301990.664414201, -3720764.479257549, 46274...","[-2301990.5737222885, -3720764.534587969, 4627..."
spacecraftId,10,10
accessId,b87fe4e3-e320-4e83-af30-b4abbfe77319,c557bcc1-fd36-48aa-8e3b-4436548be780
accessrequestId,43184453-3598-47cd-8de9-b7b760b27b4f,43184453-3598-47cd-8de9-b7b760b27b4f
tileId,951dcb65-990d-496e-95bb-6297d54f9dbb,e73a63de-f7b9-49b1-be01-cabb3e7de083
windowOpen,2023-11-14T16:19:37.893Z,2023-11-14T16:19:37.893Z
windowClose,2023-11-14T16:21:29.816Z,2023-11-14T16:21:29.816Z
windowOpenLocal,2023-11-14T08:19:37.893Z,2023-11-14T08:19:37.893Z
windowCloseLocal,2023-11-14T08:21:29.816Z,2023-11-14T08:21:29.816Z


In [14]:
# Weird, identical except for very close centerEcef values ().
df1.centerEcef[0:2].values
#& therefore different, but seemingly identical tileIds

array([list([-2301990.664414201, -3720764.479257549, 4627458.836938152]),
       list([-2301990.5737222885, -3720764.534587969, 4627458.837560752])],
      dtype=object)

In [15]:
# Drop duplicate windowOpenTime
df1.windowOpen.duplicated().sum()

41

In [16]:
tmp = df1[df1.windowOpen.duplicated()].reset_index(drop=True)

In [17]:
#tmp = df1.sort_values('windowOpen')#.reset_index(drop=True)
print(tmp.windowOpen.iloc[0])
print(tmp.windowOpen.iloc[-1])
tmp.to_csv('45_november.csv', index=False) #unnamed??
#tmp.tail()

2023-11-14T16:19:37.893Z
2023-11-30T10:45:25.119Z


In [18]:
df = pd.read_csv('45_november.csv')
print(len(df))
print(df.windowOpen.is_monotonic_increasing)
df.windowOpen.iloc[-1] #windowClose': '2023-12-11T05:29:26.000Z', NOTE access request

41
True


'2023-11-30T10:45:25.119Z'

In [21]:
# Another request 30 days from last date
start = pd.Timestamp('2023-12-01T00:00:00Z')
end = pd.Timestamp('2024-01-01T00:00:00Z')

r = submit_request(start, end)
time.sleep(5) #takes a little time to post
# Results retrieved via access request ID:
ar_id2 = r.json()['properties']['accessrequestId']
print(ar_id2)
results = requests.get("https://api.capellaspace.com/ma/accesses/" + ar_id2, headers=headers)

# View requests as table
df2 = pd.json_normalize(results.json())
print(len(df2))
df2.head()

Searching 2023-12-01T00:00:00Z To 2024-01-01T00:00:00Z
69430848-be32-4ad4-85af-481c6ea6335f
160


,center,centerEcef,spacecraftId,accessId,accessrequestId,tileId,windowOpen,windowClose,windowOpenLocal,windowCloseLocal,orbitalPlane,accessProperties.ascdsc,accessProperties.lookDirection,accessProperties.localTime,accessProperties.azimuthOpen,accessProperties.azimuthClose,accessProperties.elevationMin,accessProperties.elevationMax,accessProperties.offNadirMin,accessProperties.offNadirMax
0,"[-121.7445629415389, 46.79641971050866, 1589]","[-2301990.664414201, -3720764.479257549, 46274...",10,51150d5e-a832-4ba6-af29-bd5921f16b33,69430848-be32-4ad4-85af-481c6ea6335f,49386385-326d-44fb-a6e5-9a0f0b01aaf5,2023-12-01T09:10:21.009Z,2023-12-01T09:11:01.420Z,2023-12-01T01:10:21.009Z,2023-12-01T01:11:01.420Z,45,ascending,left,3804,167.962526,147.306544,33.286589,33.792177,49.602979,49.999914
1,"[-121.7445629415389, 46.79641971050866, 1589]","[-2301990.664414201, -3720764.479257549, 46274...",10,ca4b295a-afcc-4d30-a6ed-34bf28814090,69430848-be32-4ad4-85af-481c6ea6335f,49386385-326d-44fb-a6e5-9a0f0b01aaf5,2023-12-01T10:49:42.974Z,2023-12-01T10:52:28.090Z,2023-12-01T02:49:42.974Z,2023-12-01T02:52:28.090Z,45,ascending,left,9828,241.461270,122.967786,33.281092,57.388849,29.583960,49.998942
2,"[-121.7445629415389, 46.79641971050866, 1589]","[-2301990.664414201, -3720764.479257549, 46274...",10,99f08b17-70bb-4e86-87a0-4d6f7e94bb12,69430848-be32-4ad4-85af-481c6ea6335f,49386385-326d-44fb-a6e5-9a0f0b01aaf5,2023-12-02T09:13:44.057Z,2023-12-02T09:16:15.393Z,2023-12-02T01:13:44.057Z,2023-12-02T01:16:15.393Z,45,ascending,left,4063,204.886319,120.652511,33.289532,42.650015,42.375575,49.999807
3,"[-121.7445629415389, 46.79641971050866, 1589]","[-2301990.664414201, -3720764.479257549, 46274...",10,4a893053-a1be-45eb-9388-bc8492ee0ab0,69430848-be32-4ad4-85af-481c6ea6335f,49386385-326d-44fb-a6e5-9a0f0b01aaf5,2023-12-03T09:17:47.003Z,2023-12-03T09:20:53.752Z,2023-12-03T01:17:47.003Z,2023-12-03T01:20:53.752Z,45,ascending,left,4323,223.821088,112.224176,33.291631,51.177943,35.061568,49.998767
4,"[-121.74456155035168, 46.79641971868754, 1589]","[-2301990.5737222885, -3720764.534587969, 4627...",10,eede1c40-616b-4493-8ec1-6406a73b3277,69430848-be32-4ad4-85af-481c6ea6335f,fdb32a65-b190-4031-965a-e20485f0be00,2023-12-01T09:10:21.009Z,2023-12-01T09:11:01.420Z,2023-12-01T01:10:21.009Z,2023-12-01T01:11:01.420Z,45,ascending,left,3804,167.962535,147.306552,33.286590,33.792178,49.602978,49.999912


In [22]:
ind_dup = df2.windowOpen.duplicated()
print(f'Dropping {ind_dup.sum()} duplicates...')
df2 = df2[ind_dup].reset_index(drop=True)

Dropping 80 duplicates...


In [23]:
tmp = df2.sort_values('windowOpen')#.reset_index(drop=True)
print(tmp.windowOpen.iloc[0])
print(tmp.windowOpen.iloc[-1])

2023-12-01T09:10:21.009Z
2023-12-31T21:57:53.448Z


In [24]:
tmp.to_csv('45_december.csv', index=False)

In [25]:
# Another request 30 days from last date
start = pd.Timestamp('2024-01-01T00:00:00Z')
end = pd.Timestamp('2024-02-01T00:00:00Z')

r = submit_request(start_date=start)
time.sleep(5) #takes a little time to post
# Results retrieved via access request ID:
ar_id3 = r.json()['properties']['accessrequestId']
print(ar_id3)
results = requests.get("https://api.capellaspace.com/ma/accesses/" + ar_id3, headers=headers)

# View requests as table
df3 = pd.json_normalize(results.json())
df3.head()

Searching 2024-01-01T00:00:00Z To 2024-01-31T00:00:00Z
14800bee-0e47-4927-ac85-c99e4f284f90


,center,centerEcef,spacecraftId,accessId,accessrequestId,tileId,windowOpen,windowClose,windowOpenLocal,windowCloseLocal,orbitalPlane,accessProperties.ascdsc,accessProperties.lookDirection,accessProperties.localTime,accessProperties.azimuthOpen,accessProperties.azimuthClose,accessProperties.elevationMin,accessProperties.elevationMax,accessProperties.offNadirMin,accessProperties.offNadirMax
0,"[-121.74456289285206, 46.79641971068774, 1589]","[-2301990.661244861, -3720764.4812013116, 4627...",9,3179b34e-9a61-4568-8a9e-296b3118cc90,14800bee-0e47-4927-ac85-c99e4f284f90,a5f09711-9203-4140-b9a8-544ac57a841c,2024-01-01T20:19:35.222Z,2024-01-01T20:22:03.024Z,2024-01-01T12:19:35.222Z,2024-01-01T12:22:03.024Z,45,ascending,left,44012,204.370894,121.428383,33.348058,42.393550,42.631272,49.999979
1,"[-121.74456289285206, 46.79641971068774, 1589]","[-2301990.661244861, -3720764.4812013116, 4627...",9,c59c3dc0-c845-47d9-9895-e81d7fc14448,14800bee-0e47-4927-ac85-c99e4f284f90,a5f09711-9203-4140-b9a8-544ac57a841c,2024-01-02T20:21:09.671Z,2024-01-02T20:24:11.687Z,2024-01-02T12:21:09.671Z,2024-01-02T12:24:11.687Z,45,ascending,left,44123,222.227858,113.136586,33.349663,50.287891,35.866800,49.999666
2,"[-121.74456289285206, 46.79641971068774, 1589]","[-2301990.661244861, -3720764.4812013116, 4627...",9,618da100-1b59-4e8b-859e-e5ec5307210e,14800bee-0e47-4927-ac85-c99e4f284f90,a5f09711-9203-4140-b9a8-544ac57a841c,2024-01-03T20:22:54.638Z,2024-01-03T20:26:12.548Z,2024-01-03T12:22:54.638Z,2024-01-03T12:26:12.548Z,45,ascending,left,44236,234.793913,110.304811,33.348986,56.628074,30.291193,49.999716
3,"[-121.74456289285206, 46.79641971068774, 1589]","[-2301990.661244861, -3720764.4812013116, 4627...",9,441f798e-987e-45aa-b6b1-a16dce60e377,14800bee-0e47-4927-ac85-c99e4f284f90,a5f09711-9203-4140-b9a8-544ac57a841c,2024-01-04T18:45:05.535Z,2024-01-04T18:46:36.318Z,2024-01-04T10:45:05.535Z,2024-01-04T10:46:36.318Z,45,ascending,left,38314,183.119069,135.336209,33.318225,36.111251,47.785602,49.999843
4,"[-121.74456289285206, 46.79641971068774, 1589]","[-2301990.661244861, -3720764.4812013116, 4627...",9,0512f856-1ee9-43f1-b50c-7e55f51db659,14800bee-0e47-4927-ac85-c99e4f284f90,a5f09711-9203-4140-b9a8-544ac57a841c,2024-01-04T20:24:45.190Z,2024-01-04T20:27:05.192Z,2024-01-04T12:24:45.190Z,2024-01-04T12:27:05.192Z,45,ascending,left,44318,243.417970,137.692424,33.347915,54.651654,32.039957,49.999917


In [26]:
len(df3)

148

In [27]:
ind_dup = df3.windowOpen.duplicated()
print(f'Dropping {ind_dup.sum()} duplicates...')
df3 = df3[ind_dup].reset_index(drop=True)

Dropping 74 duplicates...


In [28]:
tmp = df3.sort_values('windowOpen')#.reset_index(drop=True)
print(tmp.windowOpen.iloc[0])
print(tmp.windowOpen.iloc[-1])

2024-01-01T20:19:35.222Z
2024-01-30T09:29:04.334Z


In [29]:
tmp.to_csv('45_january.csv', index=False)